<a href="https://colab.research.google.com/github/TheNeodev/Notebook/blob/main/RVC-MAKER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h1 align="center"><b>Ultimate RVC Maker 🎵</b></h1>
<div align="center">

A high-quality voice conversion tool focused on experimentation and performance, built upon [Vietnamese-RVC](https://github.com/PhamHuynhAnh16/Vietnamese-RVC), a fork of the original Retrieval-based Voice Conversion (RVC) project.

</div>

---

## <h2 align="center">Overview</h2>

This notebook provides a streamlined environment to set up and run the Ultimate RVC Maker, including installation, web interface, Google Drive integration, and model uploading to Hugging Face. Follow the cells below to get started.

**Resources:**
- [Support Discord](https://discord.gg/aihub)
- [GitHub Repository](https://github.com/unchCrew/RVC-MAKER.git)
- [Terms of Use](https://github.com/unchCrew/RVC-MAKER/blob/main/TERMS_OF_USE.md)

**Created by:** [TheNeoDev](https://github.com/TheNeodev)

---

## <h2 align="center">Acknowledgments</h2>

Special thanks to:
- **Original RVC Team**: For developing the core Retrieval-based Voice Conversion framework.
- **Vietnamese-RVC**: For providing an up-to-date fork used as the base for this project.

## <h2 align="center">Disclaimer</h2>

By using Ultimate RVC Maker, you agree to:
- Comply with ethical and legal standards.
- Respect intellectual property and privacy rights.
- Avoid harmful or prohibited uses.
- Accept full responsibility for any outcomes.

Ultimate RVC Maker disclaims liability and reserves the right to amend these terms. See the [Terms of Use](https://github.com/unchCrew/RVC-MAKER/blob/main/TERMS_OF_USE.md) for details.

In [ ]:
#@title **Install Ultimate RVC Maker**

import os
from IPython.display import clear_output

print("👩🏻‍💻 Starting installation...")

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Clone repository
if not os.path.exists('/content/program'):
    !git clone https://github.com/unchCrew/RVC-MAKER.git /content/program
else:
    print("Repository already cloned, skipping...")

# Install dependencies
!pip install -r /content/program/requirements.txt --quiet
!pip install pyngrok --quiet
!pip install torch --index-url https://download.pytorch.org/whl/cu128 --quiet

# Verify installation
try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
except ImportError:
    print("⚠️ Failed to import PyTorch. Installation may have issues.")

clear_output()
print("✅ Installation completed successfully! (~2 minutes)")
#@markdown **Note:** Installation typically takes about 2 minutes. If issues arise, check the output for errors.

In [ ]:
#@title **Run Web Interface**
#@markdown Launch the Ultimate RVC Maker web interface using your preferred sharing method.

import os
import threading
import time
import urllib.request
import ipywidgets as widgets
from IPython.display import display
from pyngrok import ngrok

%cd /content/program

#@markdown ### Options
#@markdown - **Enable TensorBoard**: Monitor training progress (logs saved in `./assets/logs`).
tensorboard = False #@param {type:"boolean"}

#@markdown - **Sharing Method**: Choose how to access the web interface.
method = "gradio" #@param ["gradio", "localtunnel", "ngrok"]

#@markdown - **Ngrok Token**: Required for ngrok sharing. Get it from [ngrok dashboard](https://dashboard.ngrok.com/get-started/your-authtoken).
ngrok_token = "" #@param {type:"string"}

def start_gradio():
    print("Starting Gradio interface...")
    !python main/app/app.py --share

def start_localtunnel():
    print("Starting localtunnel...")
    !npm install -g localtunnel &>/dev/null
    with open('url.txt', 'w') as file:
        file.write('')
    get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')
    time.sleep(3)
    try:
        endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
        with open('url.txt', 'r') as file:
            tunnel_url = file.read().replace("your url is: ", "").strip()
        print(f"Share Link: \033[93m{tunnel_url}\033[0m")
        display(widgets.Text(value=endpoint_ip, description='Password IP:', disabled=True))
        !python main/app/app.py
    except Exception as e:
        print(f"Error starting localtunnel: {e}")

def start_ngrok():
    if not ngrok_token:
        print("⚠️ Ngrok token is required. Please provide a valid token.")
        return
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(7860)
        print(f"Ngrok URL: \033[93m{tunnel.public_url}\033[0m")
        !python main/app/app.py --listen
    except Exception as e:
        print(f"Error starting ngrok: {e}")

def start_app():
    try:
        if method == "gradio":
            start_gradio()
        elif method == "localtunnel":
            start_localtunnel()
        elif method == "ngrok":
            start_ngrok()
    except Exception as e:
        print(f"Failed to start application: {e}")

# Start TensorBoard if enabled
if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

# Start application in a separate thread
thread_app = threading.Thread(target=start_app)
thread_app.start()

# Keep the cell running
while True:
    time.sleep(5)

#@markdown **Note:** Use the interface for full functionality. If using TensorBoard, check for overtraining in the logs.

In [ ]:
#@title **Mount Google Drive**
#@markdown Mount Google Drive to store or access files for Ultimate RVC Maker.

from google.colab import drive

try:
    drive.mount("/content/drive", force_remount=False)
    print("✅ Google Drive mounted successfully.")
except Exception as e:
    print(f"⚠️ Failed to mount Google Drive: {e}")

#@markdown **Note:** Ensure you have sufficient space in Google Drive for backups.

In [ ]:
#@title **Toggle Auto Backup**
#@markdown Enable or disable automatic backups to Google Drive. Backups save logs from `/content/program/assets/logs` to `/content/drive/MyDrive/RVCBackup`.

import os
import shutil
import time
import threading

LOGS_FOLDER = "/content/program/assets/logs"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/RVCBackup"

if "autobackups" not in globals():
    autobackups = False

#@markdown **Backup Cooldown**: Time (in seconds) between checks when files are up to date.
cooldown = 15 #@param {type:"slider", min:0, max:100, step:1}

def backup_files():
    if not os.path.exists(GOOGLE_DRIVE_PATH):
        print(f"⚠️ Google Drive not mounted at {GOOGLE_DRIVE_PATH}. Please run the 'Mount Google Drive' cell first.")
        return
    
    print("Starting backup loop...")
    last_backup_timestamps_path = os.path.join(LOGS_FOLDER, "last_backup_timestamps.txt")
    fully_updated = False

    while True:
        try:
            updated_files, deleted_files, new_files = 0, 0, 0
            last_backup_timestamps = {}

            # Load previous timestamps
            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = {k: float(v) for k, v in (line.strip().split(":", 1) for line in f)}
            except FileNotFoundError:
                pass

            # Backup new or updated files
            for root, dirs, files in os.walk(LOGS_FOLDER):
                dirs[:] = [d for d in dirs if d not in ("zips", "mute")]
                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER))
                            os.makedirs(os.path.dirname(backup_filepath), exist_ok=True)
                            current_timestamp = os.path.getmtime(filepath)
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            if last_backup_timestamp is None or last_backup_timestamp < current_timestamp:
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = current_timestamp
                                if last_backup_timestamp is None:
                                    new_files += 1
                                else:
                                    updated_files += 1

            # Remove deleted files from backup
            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER))
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        deleted_files += 1
                    del last_backup_timestamps[filepath]

            # Log backup status
            if new_files > 0 or updated_files > 0 or deleted_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            # Save timestamps
            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)

        except Exception as error:
            print(f"Error during backup: {error}")
            time.sleep(5)

# Toggle autobackup
if autobackups:
    autobackups = False
    print("✅ Autobackup disabled.")
else:
    autobackups = True
    print("✅ Autobackup enabled.")
    threading.Thread(target=backup_files, daemon=True).start()

#@markdown **Note:** Ensure Google Drive is mounted before enabling backups. The backup runs in the background and checks for changes every 0.1s (or `cooldown` seconds when up to date).

In [ ]:
#@title **Upload Model to Hugging Face**
#@markdown Upload your trained model and index file to a Hugging Face repository.

import huggingface_hub
import zipfile
import os

#@markdown **Repository ID**: Format as `username/repo_name`.
repo_hf = "NeoPy/TTS-G" #@param {type:"string"}

#@markdown **Model Path**: Path to the `.pth` model file.
pth = "/content/program/assets/weights/TTS_100e_500s.pth" #@param {type:"string"}

#@markdown **Index Path**: Path to the `.index` file.
index = "/content/program/assets/logs/TTS/added_IVF59_Flat_nprobe_1_TTS_v2.index" #@param {type:"string"}

#@markdown **Hugging Face Token**: Get it from [Hugging Face settings](https://huggingface.co/settings/tokens).
token = "hf_" #@param {type:"string"}

def upload_model(repo, pth, index, token):
    try:
        # Validate inputs
        if not repo or '/' not in repo:
            return "❌ Invalid repository ID. Use format: username/repo_name"
        if not token.startswith("hf_"):
            return "❌ Invalid Hugging Face token. It should start with 'hf_'"
        if not os.path.exists(pth):
            return f"❌ Model file not found at {pth}"
        if not os.path.exists(index):
            return f"❌ Index file not found at {index}"

        # Create README
        repo_name = repo.split('/')[-1]
        readme = f"""
# {repo_name}
This model was trained and uploaded using [Ultimate RVC Maker](https://github.com/unchCrew/RVC-MAKER) by [TheNeoDev](https://github.com/TheNeodev).
        """

        # Create zip file
        zip_path = f"{repo_name}.zip"
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            zipf.write(pth, os.path.basename(pth))
            zipf.write(index, os.path.basename(index))
            zipf.writestr('README.md', readme)

        # Upload to Hugging Face
        api = huggingface_hub.HfApi()
        api.create_repo(repo_id=repo, token=token, exist_ok=True)
        api.upload_file(
            path_or_fileobj=zip_path,
            path_in_repo=f"{repo_name}.zip",
            repo_id=repo,
            token=token
        )

        # Clean up
        os.remove(zip_path)
        return "✅ Model uploaded successfully to Hugging Face!"

    except Exception as e:
        return f"❌ Error uploading model: {e}"

# Execute upload
print(upload_model(repo_hf, pth, index, token))

#@markdown **Note:** Ensure the model and index files exist at the specified paths. The upload creates a zip file containing both files and a README.

In [ ]:
#@title **Clean Up Temporary Files**
#@markdown Remove temporary files to free up space in the Colab environment.

import os
import shutil

def clean_temp_files():
    temp_dirs = ['/content/program', '/content/url.txt']
    freed_space = 0
    for path in temp_dirs:
        try:
            if os.path.isfile(path):
                size = os.path.getsize(path) / (1024 * 1024)  # Size in MB
                os.remove(path)
                freed_space += size
                print(f"Removed file: {path}")
            elif os.path.isdir(path):
                size = sum(os.path.getsize(os.path.join(root, f)) for root, _, files in os.walk(path) for f in files) / (1024 * 1024)
                shutil.rmtree(path, ignore_errors=True)
                freed_space += size
                print(f"Removed directory: {path}")
        except Exception as e:
            print(f"Error removing {path}: {e}")
    return f"✅ Cleaned up temporary files. Freed approximately {freed_space:.2f} MB."

print(clean_temp_files())

#@markdown **Warning:** This will delete the `/content/program` directory and other temporary files. Ensure you have backed up important data.